Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('utc'))
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [3]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_backtest'

In [4]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-10T14:00:00
---------------------------------------
OHLCV
    O: 0.0964 | C: 0.0966 | V: 8929.1 | T: 2018-01-10T14:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-10T14:30:00
---------------------------------------
OHLCV
    O: 0.0966 | C: 0.0965 | V: 9865.7 | T: 2018-01-10T14:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
    ETH - {'free': -0.05, 'used': 0.0, 'total': -0.05}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 201

2018-01-10 14:00:00
Order Failed {
    "id": "b6faa3c0dcb44aa2a1ba868642a6d993",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-22T02:15:08.827635",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00483. BTC balance is: 1.0. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
Equal? False
2018-01-10 14:30:00
Order Failed {
    "id": "b6faa3c0dcb44aa2a1ba868642a6d993",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-22T02:15:08.827635",
    "opened_tim

---------------------------------------
Epoch 12 - Timestep: 2018-01-10T19:30:00
---------------------------------------
OHLCV
    O: 0.0886 | C: 0.0909 | V: 6753.6 | T: 2018-01-10T19:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0909 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9496 Total Val: 0.9983 PnL: -0.0017 Returns: -0.0017
BALANCE
    BTC - {'free': 0.98636550000000001, 'used': 0.0, 'total': 0.98636550000000001}
    ETH - {'free': -0.25, 'used': 0.0, 'total': -0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.5499999999999999, 'cost_price': 0.091649636363636378, 'latest_price': 0.088610000000000008}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 13 - Timestep: 2018-01-10T20:00:00
---------------------------------------
OHLCV
    O: 0.0908 | C: 0.0901 | V: 6086.7 | T: 2018-01-10T20:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9450 Total Val: 0.9996 PnL: -0.0004 Returns: -0.0004
B

2018-01-10 19:30:00
Equal? False
2018-01-10 20:00:00
Equal? False
2018-01-10 20:30:00
Equal? False
2018-01-10 21:00:00
Equal? False
2018-01-10 21:30:00
Equal? False
2018-01-10 22:00:00
Order Failed {
    "id": "c12ca9def4f5406d8722e67e6d38743b",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-22T02:15:09.126186",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00433. BTC balance is: 0.99966525. ETH balance is 1.3877787807814457e-17"
    },
    "symbol": "ETH/BTC"
}
Equal? False
2018-01-10 22:30:00
Order Failed {
    "id": "c12ca9def4f5406d8722e67e6d38743b",
    "exchange_id": "paper",
    "exchange_order_id": null,

---------------------------------------
Epoch 22 - Timestep: 2018-01-11T00:30:00
---------------------------------------
OHLCV
    O: 0.0881 | C: 0.0915 | V: 9052.8 | T: 2018-01-11T00:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0915 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9102 Total Val: 0.9982 PnL: -0.0018 Returns: -0.0018
BALANCE
    BTC - {'free': 0.99986039999999998, 'used': 0.0, 'total': 0.99986039999999998}
    ETH - {'free': -0.7500000000000001, 'used': 0.0, 'total': -0.7500000000000001}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.0000000000000002, 'cost_price': 0.089795099999999989, 'latest_price': 0.088002999999999998}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 23 - Timestep: 2018-01-11T01:00:00
---------------------------------------
OHLCV
    O: 0.0915 | C: 0.0896 | V: 8795.0 | T: 2018-01-11T01:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0896 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9056 Total Val: 1.0017 PnL: 

...

2018-01-11 13:30:00
Equal? False
2018-01-11 14:00:00
Equal? False
2018-01-11 14:30:00
Equal? False
2018-01-11 15:00:00
Equal? False
2018-01-11 15:30:00
Equal? False
2018-01-11 16:00:00
Equal? False
2018-01-11 16:30:00
Equal? False
2018-01-11 17:00:00
Equal? False


---------------------------------------
Epoch 56 - Timestep: 2018-01-11T17:30:00
---------------------------------------
OHLCV
    O: 0.0888 | C: 0.0893 | V: 6388.7 | T: 2018-01-11T17:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0893 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.7587 Total Val: 0.9987 PnL: -0.0013 Returns: -0.0013
BALANCE
    BTC - {'free': 0.98221469999999977, 'used': 0.0, 'total': 0.98221469999999977}
    ETH - {'free': -1.2000000000000004, 'used': 0.0, 'total': -1.2000000000000004}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 2.6999999999999984, 'cost_price': 0.089361629629629652, 'latest_price': 0.088882000000000003}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 57 - Timestep: 2018-01-11T18:00:00
---------------------------------------
OHLCV
    O: 0.0893 | C: 0.0893 | V: 7204.2 | T: 2018-01-11T18:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.7543 Total Val: 1.0000 PnL

2018-01-11 17:30:00
Equal? False
2018-01-11 18:00:00
Equal? False
2018-01-11 18:30:00
Equal? False
2018-01-11 19:00:00
Equal? False
2018-01-11 19:30:00
Equal? False
2018-01-11 20:00:00
Equal? False
2018-01-11 20:30:00
Equal? False
2018-01-11 21:00:00


    RSI: 10.00
---------------------------------------
Epoch 64 - Timestep: 2018-01-11T21:30:00
---------------------------------------
OHLCV
    O: 0.0879 | C: 0.0876 | V: 4851.5 | T: 2018-01-11T21:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.7234 Total Val: 0.9956 PnL: -0.0044 Returns: -0.0044
BALANCE
    BTC - {'free': 0.98225994999999966, 'used': 0.0, 'total': 0.98225994999999966}
    ETH - {'free': -1.2000000000000004, 'used': 0.0, 'total': -1.2000000000000004}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 3.099999999999997, 'cost_price': 0.089231048387096834, 'latest_price': 0.087802999999999992}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 65 - Timestep: 2018-01-11T22:00:00
---------------------------------------
OHLCV
    O: 0.0876 | C: 0.0880 | V: 5223.5 | T: 2018-01-11T22:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.7190 Total

Equal? False
2018-01-11 21:30:00
Equal? False
2018-01-11 22:00:00
Equal? False
2018-01-11 22:30:00
Equal? False
2018-01-11 23:00:00
Equal? False
2018-01-11 23:30:00
Equal? False
No data after prior poll: 2018-01-11 23:30:00


## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(c.ETH, c.BTC), Asset(c.LTC, c.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)